In [1]:
# required modules 
import pandas as pd
import re
from konlpy.tag import Okt
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
# sample dataframe 
data = {
    'name': [
        '제주 갈치조림', 
        '부산 돼지국밥', 
        '서울 떡볶이', 
        '강릉 초당순두부', 
        '전주 비빔밥'
    ],
    'raw_text': [
        '사장님이 너무 친절하고 갈치가 매콤해서 맛있었어요',
        '국물이 진하고 고기가 부드러워서 정말 좋았어요',
        '떡이 쫄깃하고 양념이 맛있어서 또 가고 싶어요',
        '담백하고 고소한 순두부가 일품이었어요',
        '비빔밥 재료가 신선하고 양이 많아서 만족했어요'
    ]
}
df = pd.DataFrame(data)


# data_path = ''
# df = pd.read_csv(data_path)

# text_processing

In [4]:
okt = Okt()

def remove_non_korean(text):
    """리뷰에서 한글을 제외한 모든 문자 제거"""
    return re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣\s]", "", text)


def load_stopwords(filepath="k_stopword.txt"):
    """불용어 txt 파일을 읽어 불용어 리스트 반환"""
    with open(filepath, "r", encoding="utf-8") as f:
        stopwords = f.read().splitlines()
    return stopwords

stopwords = load_stopwords()


def okt_tokenize(text):
    """Okt를 이용한 형태소 분석 및 불필요 품사 제거"""
    # 형태소 분석 및 품사 태깅
    words = okt.pos(text, stem=True)  # 어간 추출
    
    # 필요한 품사만 필터링 (명사, 동사, 형용사, 부사)
    selected_words = [
        word for word, pos in words if pos in ['Noun', 'Verb', 'Adjective', 'Adverb']
    ]
    
    # 불용어 제거
    filtered_words = [word for word in selected_words if word not in stopwords]
    
    # 최종 단어들을 문자열로 반환
    return " ".join(filtered_words)


def preprocess_review(text):
    """리뷰 텍스트를 전처리하고 형태소 분석한 최종 결과 반환"""
    text = remove_non_korean(text)
    processed_text = okt_tokenize(text)
    
    return processed_text

In [5]:
tqdm.pandas()
df['pos_tag'] = df['raw_text'].progress_apply(preprocess_review)

100%|██████████| 5/5 [00:05<00:00,  1.00s/it]


In [6]:
df

,name,raw_text,pos_tag
0,제주 갈치조림,사장님이 너무 친절하고 갈치가 매콤해서 맛있었어요,사장 너무 친절하다 갈치 매콤 맛있다
1,부산 돼지국밥,국물이 진하고 고기가 부드러워서 정말 좋았어요,국물 진하다 고기 부드럽다 정말 좋다
2,서울 떡볶이,떡이 쫄깃하고 양념이 맛있어서 또 가고 싶어요,떡 쫄깃 양념 맛있다 가다 싶다
3,강릉 초당순두부,담백하고 고소한 순두부가 일품이었어요,담백하다 고소하다 순두부 일품
4,전주 비빔밥,비빔밥 재료가 신선하고 양이 많아서 만족했어요,비빔밥 재료 신선하다 양 많다 만족하다


# Keyword Extraction by TF-IDF 

In [11]:
# TF-IDF 키워드 추출 함수
def extract_keywords_tfidf(corpus, top_n):
    """TF-IDF를 사용해 각 문서의 상위 N개 키워드 추출"""
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(corpus)
    features = vectorizer.get_feature_names_out()
    
    keywords = []
    for i in tqdm(range(X.shape[0])):
        row = X[i].toarray().flatten()
        top_indices = row.argsort()[-top_n:][::-1]
        top_keywords = [features[idx] for idx in top_indices]
        keywords.append(top_keywords)
    return keywords

In [12]:
# pos_tag 열에서 키워드 추출
df['keywords'] = extract_keywords_tfidf(df['pos_tag'].tolist(),3)

100%|██████████| 5/5 [00:00<00:00, 2153.58it/s]


In [13]:
df

,name,raw_text,pos_tag,keywords
0,제주 갈치조림,사장님이 너무 친절하고 갈치가 매콤해서 맛있었어요,사장 너무 친절하다 갈치 매콤 맛있다,"[친절하다, 너무, 갈치]"
1,부산 돼지국밥,국물이 진하고 고기가 부드러워서 정말 좋았어요,국물 진하다 고기 부드럽다 정말 좋다,"[진하다, 정말, 좋다]"
2,서울 떡볶이,떡이 쫄깃하고 양념이 맛있어서 또 가고 싶어요,떡 쫄깃 양념 맛있다 가다 싶다,"[쫄깃, 양념, 싶다]"
3,강릉 초당순두부,담백하고 고소한 순두부가 일품이었어요,담백하다 고소하다 순두부 일품,"[일품, 담백하다, 고소하다]"
4,전주 비빔밥,비빔밥 재료가 신선하고 양이 많아서 만족했어요,비빔밥 재료 신선하다 양 많다 만족하다,"[재료, 만족하다, 많다]"
